# Иерархическая индексация

Библиотека Pandas предоставляет объекты Panel и Panel4D, позволяющие нативным образом хранить трехмерные и четырехмерные данные. На практике намного чаще используется иерархическая индексация (hierarchical indexing), или мультииндексация (multi-indexing), для включения в один индекс нескольких уровней. При этом многомерные данные могут быть компактно представлены в уже привычных нам одномерных объектах Series и двумерных объектах DataFrame.

In [1]:
import numpy as np
import pandas as pd

## Мультииндексированный объект Series

Рассмотрим, как можно представить двумерные данные в одномерном объекте Series. Для конкретики изучим ряд данных, в котором у каждой точки имеются символьный и числовой ключи.

### Плохой способ

Нам требуется проанализировать данные о штатах за два разных года.

In [2]:
# применим в качестве ключей кортежи языка Python
index = [('California', 2000), ('California', 2010), ('New York', 2000), ('New York', 2010), ('Texas', 2000), ('Texas', 2010)]
populations = [33871648, 37253956, 18976457, 19378102, 20851820, 25145561]
pop = pd.Series(populations, index=index)
pop

(California, 2000)    33871648
(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
(Texas, 2010)         25145561
dtype: int64

При подобной схеме индексации появляется возможность непосредственно индексировать или выполнять срез ряда данных на основе такого мультииндекса:

In [3]:
pop[('California', 2010):('Texas', 2000)]

(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
dtype: int64

Например, при необходимости выбрать все значения из 2010 года придется проделать громоздкую (и потенциально медленную) очистку данных:

In [4]:
pop[[i for i in pop.index if i[1] == 2010]]

(California, 2010)    37253956
(New York, 2010)      19378102
(Texas, 2010)         25145561
dtype: int64

Это хоть и приводит к желаемому результату, но гораздо менее изящно (и далеко не так эффективно), как использование синтаксиса срезов Pandas

### Лучший способ

Наша индексация, основанная на кортежах, по сути, является примитивным мультииндексом, и тип MultiIndex библиотеки Pandas как раз обеспечивает необходимые нам операции. Создать мультииндекс из кортежей можно следующим образом:

In [5]:
index = pd.MultiIndex.from_tuples(index)
index

MultiIndex(levels=[['California', 'New York', 'Texas'], [2000, 2010]],
           codes=[[0, 0, 1, 1, 2, 2], [0, 1, 0, 1, 0, 1]])

Проиндексировав заново наши ряды данных с помощью MultiIndex, мы увидим иерархическое представление данных:

In [6]:
pop = pop.reindex(index)
pop

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

Здесь первые два столбца представления объекта Series отражают значения мультииндекса, а третий столбец — данные. В первом столбце отсутствуют некоторые элементы: в этом мультииндексном представлении все пропущенные элементы означают то же значение, что и строкой выше.

Теперь для выбора всех данных, второй индекс которых равен 2010, можно просто воспользоваться синтаксисом срезов библиотеки Pandas:

In [7]:
pop[:, 2010]

California    37253956
New York      19378102
Texas         25145561
dtype: int64

Результат представляет собой массив с одиночным индексом и только теми ключами, которые нас интересуют. Такой синтаксис намного удобнее (а операция выполняется гораздо быстрее!), чем мультииндексное решение на основе кортежей, с которого мы начали.

### Мультииндекс как дополнительная система

Мы могли с легкостью хранить те же самые данные с помощью простого объекта DataFrame с индексом и метками столбцов. На самом деле библиотека Pandas создана с учетом этой равнозначности. Метод unstack() может быстро преобразовать мультииндексный объект Series в индексированный обычным образом объект DataFrame:

In [8]:
pop_df = pop.unstack()
pop_df

,2000,2010
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


Как и можно ожидать, метод stack() выполняет противоположную операцию:

In [9]:
pop_df.stack()

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

Например, нам может понадобиться добавить в демографические данные по каждому штату за каждый год еще один столбец (допустим, количество населения младше 18 лет). Благодаря типу MultiIndex это сводится к добавлению еще одного столбца в объект DataFrame:

In [10]:
pop_df = pd.DataFrame({'total': pop, 'under18': [9267089, 9284094, 4687374, 4318033, 5906301, 6879014]})
pop_df

total  under18
California 2000  33871648  9267089
           2010  37253956  9284094
New York   2000  18976457  4687374
           2010  19378102  4318033
Texas      2000  20851820  5906301
           2010  25145561  6879014

In [11]:
# вычисляем по годам долю населения младше 18 лет на основе вышеприведенных данных:
f_u18 = (pop_df['under18'] / pop_df['total']) * 100
f_u18

California  2000    27.359428
            2010    24.921096
New York    2000    24.700997
            2010    22.283054
Texas       2000    28.325110
            2010    27.356773
dtype: float64

Это дает нам возможность легко и быстро манипулировать даже многомерными данными и исследовать их.

## Методы создания мультииндекса

Наиболее простой метод создания мультииндексированного объекта Series или DataFrame — передать в конструктор список из двух или более индексных массивов. Например:

In [12]:
df = pd.DataFrame(np.random.rand(4, 2), index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]], columns=['data1', 'data2'])
df

data1     data2
a 1  0.791251  0.215210
  2  0.813363  0.250611
b 1  0.743632  0.026499
  2  0.660601  0.375471

Если передать словарь с соответствующими кортежами в качестве ключей, библиотека Pandas автоматически распознает такой синтаксис и будет по умолчанию использовать мультииндекс:

In [13]:
data = {('California', 2000): 33871648,
        ('California', 2010): 37253956,
        ('Texas', 2000): 20851820,
        ('Texas', 2010): 25145561,
        ('New York', 2000): 18976457,
        ('New York', 2010): 19378102}
pd.Series(data)

California  2000    33871648
            2010    37253956
Texas       2000    20851820
            2010    25145561
New York    2000    18976457
            2010    19378102
dtype: int64

Тем не менее иногда бывает удобно создавать объекты MultiIndex явным образом. Далее мы рассмотрим несколько методов для этого.

### Явные конструкторы для объектов MultiIndex

При формировании индекса для большей гибкости можно воспользоваться имеющимися в классе pd.MultiIndex конструкторами-методами класса. Например, можно сформировать объект MultiIndex из простого списка массивов, задающих значения индекса в каждом из уровней:

In [14]:
pd.MultiIndex.from_arrays([['a', 'a', 'b', 'b'], [1, 2, 1, 2]])

MultiIndex(levels=[['a', 'b'], [1, 2]],
           codes=[[0, 0, 1, 1], [0, 1, 0, 1]])

Или из списка кортежей, задающих все значения индекса в каждой из точек:

In [15]:
pd.MultiIndex.from_tuples([('a', 1), ('a', 2), ('b', 1), ('b', 2)])

MultiIndex(levels=[['a', 'b'], [1, 2]],
           codes=[[0, 0, 1, 1], [0, 1, 0, 1]])

Или из декартова произведения обычных индексов:

In [16]:
pd.MultiIndex.from_product([['a', 'b'], [1, 2]])

MultiIndex(levels=[['a', 'b'], [1, 2]],
           codes=[[0, 0, 1, 1], [0, 1, 0, 1]])

Можно сформировать объект MultiIndex непосредственно с помощью его внутреннего представления, передав в конструктор levels (список списков, содержащих имеющиеся значения индекса для каждого уровня) и labels (список списков меток):

In [17]:
pd.MultiIndex(levels=[['a', 'b'], [1, 2]], codes=[[0, 0, 1, 1], [0, 1, 0, 1]])

MultiIndex(levels=[['a', 'b'], [1, 2]],
           codes=[[0, 0, 1, 1], [0, 1, 0, 1]])

Любой из этих объектов можно передать в качестве аргумента метода index при создании объектов Series или DataFrame или методу reindex уже существующих объектов Series или DataFrame.

### Названия уровней мультииндексов

Иногда бывает удобно задать названия для уровней объекта MultiIndex. Сделать это можно, передав аргумент names любому из вышеперечисленных конструкторов класса MultiIndex или задав значения атрибута names постфактум:

In [18]:
pop

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [19]:
pop.index.names = ['state', 'year']
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [20]:
pop.index.names

FrozenList(['state', 'year'])

### Мультииндекс для столбцов

В объектах DataFrame строки и столбцы полностью симметричны, и у столбцов, точно так же, как и у строк, может быть несколько уровней индексов. Рассмотрим следующий пример, представляющий собой имитацию неких (в чем-то достаточно реалистичных) медицинских данных:

In [21]:
# иерархические индексы и столбцы
index = pd.MultiIndex.from_product([[2013, 2014], [1, 2]], names=['year', 'visit'])
columns = pd.MultiIndex.from_product([['Bob', 'Guido', 'Sue'], ['HR', 'Temp']], names=['subject', 'type'])

# создаем имитационные данные
data = np.round(np.random.randn(4, 6), 1)
data

array([[ 0.1,  1.5,  1.9, -1.1, -0.4,  1.9],
       [-0.3,  0.8,  0.3,  0.1,  0. , -0.8],
       [ 0.8,  1. ,  0. ,  0.7, -1.5, -0.8],
       [ 0.5, -0.3,  0.4,  0.6, -2.5,  1. ]])

In [22]:
data[:, ::2] *= 10
data

array([[  1. ,   1.5,  19. ,  -1.1,  -4. ,   1.9],
       [ -3. ,   0.8,   3. ,   0.1,   0. ,  -0.8],
       [  8. ,   1. ,   0. ,   0.7, -15. ,  -0.8],
       [  5. ,  -0.3,   4. ,   0.6, -25. ,   1. ]])

In [23]:
data += 37
data

array([[38. , 38.5, 56. , 35.9, 33. , 38.9],
       [34. , 37.8, 40. , 37.1, 37. , 36.2],
       [45. , 38. , 37. , 37.7, 22. , 36.2],
       [42. , 36.7, 41. , 37.6, 12. , 38. ]])

In [24]:
# создаем объект DataFrame
health_data = pd.DataFrame(data, index=index, columns=columns)
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      38.0  38.5  56.0  35.9  33.0  38.9
     2      34.0  37.8  40.0  37.1  37.0  36.2
2014 1      45.0  38.0  37.0  37.7  22.0  36.2
     2      42.0  36.7  41.0  37.6  12.0  38.0

In [25]:
# теперь можно индексировать столбец верхнего уровня по имени человека и получить объект DataFrame, 
# содержащий информацию только об этом человеке:
health_data['Guido']

type          HR  Temp
year visit            
2013 1      56.0  35.9
     2      40.0  37.1
2014 1      37.0  37.7
     2      41.0  37.6

## Индексация и срезы по мультииндексу

### Мультииндексация объектов Series

In [26]:
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [27]:
# обращаться к отдельным элементам можно путем индексации с помощью нескольких термов:
pop['California', 2000]

33871648

Объект MultiIndex поддерживает также частичную индексацию (partial indexing), то есть индексацию только по одному из уровней индекса. Результат — тоже объект Series, с более низкоуровневыми индексами:

In [28]:
pop['California']

year
2000    33871648
2010    37253956
dtype: int64

In [29]:
pop.loc['California':'New York']

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
dtype: int64

In [30]:
# С помощью отсортированных индексов можно выполнять частичную индексацию по нижним уровням, 
# указав пустой срез в первом индексе:
pop[:, 2000]

state
California    33871648
New York      18976457
Texas         20851820
dtype: int64

In [31]:
# выборка данных на основе булевой маски
pop[pop>22000000]

state       year
California  2000    33871648
            2010    37253956
Texas       2010    25145561
dtype: int64

In [32]:
# выборка на основе прихотливой индексации
pop[['California', 'Texas']]

state       year
California  2000    33871648
            2010    37253956
Texas       2000    20851820
            2010    25145561
dtype: int64

### Мультииндексация объектов DataFrame

In [33]:
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      38.0  38.5  56.0  35.9  33.0  38.9
     2      34.0  37.8  40.0  37.1  37.0  36.2
2014 1      45.0  38.0  37.0  37.7  22.0  36.2
     2      42.0  36.7  41.0  37.6  12.0  38.0

В объектах DataFrame основными являются столбцы, и используемый для мультииндексированных Series синтаксис применяется тоже к столбцам. Например, можно узнать пульс Гвидо с помощью простой операции:

In [34]:
health_data['Guido', 'HR']

year  visit
2013  1        56.0
      2        40.0
2014  1        37.0
      2        41.0
Name: (Guido, HR), dtype: float64

In [35]:
# Как и в случае с одиночным индексом, можно использовать индексаторы loc, iloc и ix
health_data.iloc[:2, :2]

subject      Bob      
type          HR  Temp
year visit            
2013 1      38.0  38.5
     2      34.0  37.8

Эти индексаторы возвращают массивоподобное представление исходных двумерных данных, но в каждом отдельном индексе в loc и iloc можно указать кортеж из нескольких индексов. Например:

In [36]:
health_data.loc[:, ('Bob', 'HR')]

year  visit
2013  1        38.0
      2        34.0
2014  1        45.0
      2        42.0
Name: (Bob, HR), dtype: float64

Работать со срезами в подобных кортежах индексов не очень удобно: попытка создать срез в кортеже может привести к синтаксической ошибке:

In [37]:
health_data.loc[(:, 1), (:, 'HR')]

SyntaxError: invalid syntax (<ipython-input-37-fb34fa30ac09>, line 1)

Избежать этого можно, сформировав срез явным образом с помощью встроенной функции Python slice(), но лучше в данном случае использовать объект IndexSlice, предназначенный библиотекой Pandas как раз для подобной ситуации. Например:

In [38]:
idx = pd.IndexSlice
health_data.loc[idx[:, 1], idx[:, 'HR']]

,subject,Bob,Guido,Sue
,type,HR,HR,HR
year,visit,,,
2013,1,38.0,56.0,33.0
2014,1,45.0,37.0,22.0


Существует множество способов взаимодействия с данными в мультииндексированных объектах Series и DataFrame, и лучший способ привыкнуть к ним — начать с ними экспериментировать!

## Перегруппировка мультииндексов

### Отсортированные и неотсортированные индексы

Большинство операций срезов с мультииндексами завершится ошибкой, если индекс не отсортирован.

In [40]:
index = pd.MultiIndex.from_product([['a', 'c', 'b'], [1, 2]])
data = pd.Series(np.random.rand(6), index=index)
data.index.names = ['char', 'int']
data

char  int
a     1      0.324242
      2      0.146596
c     1      0.918965
      2      0.625197
b     1      0.939947
      2      0.687806
dtype: float64

In [41]:
try:
    data['a':'b']
except KeyError as e:
    print(type(e))
    print(e)

<class 'pandas.errors.UnsortedIndexError'>
'Key length (1) was greater than MultiIndex lexsort depth (0)'


По различным причинам частичные срезы и другие подобные операции требуют, чтобы уровни мультииндекса были отсортированы (лексикографически упорядочены). Библиотека Pandas предоставляет множество удобных инструментов для выполнения подобной сортировки. В качестве примеров можем указать методы sort_index() и sortlevel() объекта DataFrame.

In [43]:
data = data.sort_index()
data

char  int
a     1      0.324242
      2      0.146596
b     1      0.939947
      2      0.687806
c     1      0.918965
      2      0.625197
dtype: float64

In [50]:
data['a':'b']

char  int
a     1      0.324242
      2      0.146596
b     1      0.939947
      2      0.687806
dtype: float64

### Выполнение над индексами операций stack и unstack

In [57]:
pop.unstack(level=0)

state,California,New York,Texas
year,,,
2000,33871648,18976457,20851820
2010,37253956,19378102,25145561


In [55]:
pop.unstack(level=1)

year,2000,2010
state,,
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


In [56]:
# Методу unstack() противоположен по действию метод stack(), 
# которым можно воспользоваться, чтобы получить обратно исходный ряд данных:
pop.unstack().stack()

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

### Создание и перестройка индексов

Еще один способ перегруппировки иерархических данных — преобразовать метки индекса в столбцы с помощью метода reset_index. Результатом вызова этого метода для нашего ассоциативного словаря населения будет объект DataFrame со столбцами state (штат) и year (год), содержащими информацию, ранее находившуюся в индексе. Для большей ясности можно при желании задать название для представленных в виде столбцов данных:

In [61]:
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [58]:
pop_flat = pop.reset_index(name='population')
pop_flat

,state,year,population
0,California,2000,33871648
1,California,2010,37253956
2,New York,2000,18976457
3,New York,2010,19378102
4,Texas,2000,20851820
5,Texas,2010,25145561


При работе с реальными данными исходные входные данные очень часто выглядят подобным образом, поэтому удобно создать объект MultiIndex из значений столбцов. Это можно сделать с помощью метода set_index объекта DataFrame, возвращающего мультииндексированный объект DataFrame:

In [62]:
pop_flat.set_index(['state', 'year'])

population
state      year            
California 2000    33871648
           2010    37253956
New York   2000    18976457
           2010    19378102
Texas      2000    20851820
           2010    25145561

### Агрегирование по мультииндексам

В библиотеке Pandas имеются встроенные методы для агрегирования данных, например mean(), sum() и max(). В случае иерархически индексированных данных им можно передать параметр level для указания подмножества данных, на котором будет вычисляться сводный показатель.

In [63]:
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      38.0  38.5  56.0  35.9  33.0  38.9
     2      34.0  37.8  40.0  37.1  37.0  36.2
2014 1      45.0  38.0  37.0  37.7  22.0  36.2
     2      42.0  36.7  41.0  37.6  12.0  38.0

Допустим, нужно усреднить результаты измерений показателей по двум визитам в течение года. Сделать это можно путем указания уровня индекса, который мы хотели бы исследовать, в данном случае года (year):

In [64]:
data_mean = health_data.mean(level='year')
data_mean

subject   Bob        Guido          Sue       
type       HR   Temp    HR   Temp    HR   Temp
year                                          
2013     36.0  38.15  48.0  36.50  35.0  37.55
2014     43.5  37.35  39.0  37.65  17.0  37.10

Далее, воспользовавшись ключевым словом axis, можно получить и среднее значение по уровням по столбцам:

In [69]:
data_mean.mean(axis=1, level='type')

type,HR,Temp
year,,
2013,39.666667,37.400000
2014,33.166667,37.366667


Так, всего двумя строками кода мы смогли найти средний пульс и температуру по всем субъектам и визитам за каждый год. Такой синтаксис представляет собой сокращенную форму функциональности GroupBy.